# Otimização:  Método dos Gradiente Conjugados

*Descrição da Tarefa:*

Implementar o método dos *Gradientes Conjugados Não-Linear* e *Método de Descenso Coordenado Sequencial* e utilizá-los para minimizar funções da forma:



$$f(x) = e^{\sum\limits_{i=1}^n x_i} + \sum\limits_{i=1}^na_i x_i^2$$

para diferentes valores de $a_i > 0$

### **Funções Auxiliares**

Vamos definir algumas funções que serão utilizadas posteriormente para facilitar os cálculos do *método gradiente*.

In [1]:
import numpy as np
import time

e_values = lambda x: np.ones(len(x))* np.exp(np.sum(x))
gradient = lambda x,a: 2*x*a + e_values(x) # retorna lista com os gradientes dado ponto (x,y)
apply_func = lambda x,a: np.dot(np.square(x), a) + np.exp(np.sum(x)) # f(x)
close_to_zero = lambda grad, tol: np.linalg.norm(grad,2) < tol # stop condition (euclidian distance)
hess = lambda x,a: 2*a + e_values(x)
max_L = lambda x,a: max(hess(x,a))
armijo lambda x, d, step, a, grad, c1: apply_func(x + (d*step),a) >= apply_func(x,a) + c1*step*np.dot(grad, d) 

### Funções Principais

1) Gradiente Conjugado não-linear pelo método de Fletcher-Reeves: 

In [2]:
def fletcher_reeves(x, a, tolerance=0.00000001, verbose=False, alpha=0.1, backtracking=False, c1 = 0.3, max_it=10000):
    
    grad = gradient(x,a)
    d = -1*grad
    beta = 0
    it = 0

    while((close_to_zero(grad,tolerance) == False) and it < max_it):
        
        # Choose step size
        step = alpha
        if(backtracking == True):
            while(armijo(x,d, step, a, grad, c1) == True):
                step = step/2
    
        # Update it
        x = x + (d*step)
        old_grad = grad
        grad = gradient(x,a)
        beta = np.dot(grad, grad)/np.dot(old_grad, old_grad)
        d = -1*grad + beta*d
        it = it + 1
        
    if(verbose == True):
        print('Iteracoes utilizadas = ', it)
        print('f(x) final = ', apply_func(x,a))
        
    return(x)

2) Método de Descenso Coordenado Sequencial:

In [3]:
def descenso_coordenado(x, a, tolerance=0.0001, verbose=False):
    
    grad = gradient(x,a)
    it = 0

    while((close_to_zero(grad,tolerance) == False)):
        
        index = (it%len(x))
        alpha_k = 1/max_L(x,a)
        e_k = np.zeros(len(x))
        e_k[index] = grad[index]
        x = x - (alpha_k*e_k)
        grad = gradient(x,a)
        it = it + 1
        
    if(verbose == True):
        print('Iteracoes utilizadas = ', it)
        print('f(x) final = ', apply_func(x,a))
        
    return(x)

## Experimentos

### Testar o algoritmo para funções de duas variáveis

**i) Com maior $a_i$ igual a 1.1 vezes o menor**

Fletcher-Reeves:

In [4]:
a = np.float64([1,1.1]) # temos, por exemplo, esses valores de a > 0
x = np.float64([1,2]) # e um chute inicial dos valores de xi
start_time = time.clock()
convergence_x = fletcher_reeves(x, a, tolerance=0.0001, verbose=True, backtracking=True, alpha=10)
total_time = time.clock() - start_time
print('Tempo de processamento: {:.4f} segundos'.format(total_time))
print("Ponto de mínimo encontrado: ", convergence_x)

Iteracoes utilizadas =  8
f(x) final =  0.7354020140405326
Tempo de processamento: 0.0155 segundos
Ponto de mínimo encontrado:  [-0.288352   -0.26212613]


Descenco Coordenado:

In [5]:
a = np.float64([1,1.1]) # temos, por exemplo, esses valores de a > 0
x = np.float64([1,2]) # e um chute inicial dos valores de xi
start_time = time.clock()
convergence_x = descenso_coordenado(x, a, tolerance=0.0001, verbose=True)
total_time = time.clock() - start_time
print('Tempo de processamento: {:.4f} segundos'.format(total_time))
print("Ponto de mínimo encontrado: ", convergence_x)

Iteracoes utilizadas =  13
f(x) final =  0.7354020142939202
Tempo de processamento: 0.0070 segundos
Ponto de mínimo encontrado:  [-0.28834959 -0.26211023]


**Análise:** Podemos observar que apesar do fletcher-reeves convergir com menos iterações (8 iterações), o tempo necessário para a computação dessa convergência é superior a do descenso coordenado que utiliza mais iterações (13 iterações).

**ii) Com maior $a_i$ igual a 100 vezes o menor**

Fletcher-Reeves

In [6]:
a = np.float64([1,100]) # temos, por exemplo, esses valores de a > 0
x = np.float64([1,2]) # e um chute inicial dos valores de xi
start_time = time.clock()
convergence_x = fletcher_reeves(x, a, tolerance=0.0001, verbose=True, backtracking=True, alpha=1)
total_time = time.clock() - start_time
print('Tempo de processamento: {:.4f} segundos'.format(total_time))
print("Ponto de mínimo encontrado: ", convergence_x)

Iteracoes utilizadas =  17
f(x) final =  0.8259500689732878
Tempo de processamento: 0.0120 segundos
Ponto de mínimo encontrado:  [-0.35081482 -0.00350815]


Descenco Coordenado:

In [7]:
a = np.float64([1,100]) # temos, por exemplo, esses valores de a > 0
x = np.float64([1,2]) # e um chute inicial dos valores de xi
start_time = time.clock()
convergence_x = descenso_coordenado(x, a, tolerance=0.0001, verbose=True)
total_time = time.clock() - start_time
print('Tempo de processamento: {:.4f} segundos'.format(total_time))
print("Ponto de mínimo encontrado: ", convergence_x)

Iteracoes utilizadas =  1515
f(x) final =  0.8259500707521492
Tempo de processamento: 0.0837 segundos
Ponto de mínimo encontrado:  [-0.35078475 -0.00350835]


**Análise:** Já para esta função, podemos observar que a diferença de iterações necessárias para convergir é muito maior, o que também é refletido pelo tempo de processamento utilizado em cada técnica. Dessa forma, podemos afirmar que há evidências da técnica de fletcher-reeves funcionar melhor do que o descenso coordenado, principalmente para autovalores mais discrepantes.

### Testando valores de n

**n = 3**
- Fletcher-Reeves

In [8]:
a = np.float64(range(3)) # temos, por exemplo, esses valores de a > 0
x = np.float64(range(3)) # e um chute inicial dos valores de xi
start_time = time.clock()
fletcher_reeves(x, a, tolerance=0.0001, verbose=True, backtracking=True, alpha=1)
total_time = time.clock() - start_time
print('Tempo = ', total_time)

Iteracoes utilizadas =  98
f(x) final =  1.9400140393958273e-07
Tempo =  0.021430965566783367


- Descenso Coordenado

In [9]:
a = np.float64(range(3)) # temos, por exemplo, esses valores de a > 0
x = np.float64(range(3)) # e um chute inicial dos valores de xi
start_time = time.clock()
descenso_coordenado(x, a, tolerance=0.0001, verbose=True)
total_time = time.clock() - start_time
print('Tempo = ', total_time)

Iteracoes utilizadas =  120049
f(x) final =  0.00010000257822731737
Tempo =  6.351482522468709


**n = 4**
- Fletcher-Reeves

In [10]:
a = np.float64(range(4)) # temos, por exemplo, esses valores de a > 0
x = np.float64(range(4)) # e um chute inicial dos valores de xi
start_time = time.clock()
fletcher_reeves(x, a, tolerance=0.0001, verbose=True, backtracking=True, alpha=1)
total_time = time.clock() - start_time
print('Tempo = ', total_time)

Iteracoes utilizadas =  226
f(x) final =  1.7483189430461824e-05
Tempo =  0.03560613923850653


- Descenso Coordenado

In [11]:
a = np.float64(range(4)) # temos, por exemplo, esses valores de a > 0
x = np.float64(range(4)) # e um chute inicial dos valores de xi
start_time = time.clock()
descenso_coordenado(x, a, tolerance=0.0001, verbose=True)
total_time = time.clock() - start_time
print('Tempo = ', total_time)

Iteracoes utilizadas =  240073
f(x) final =  0.00010000309802756798
Tempo =  10.430672233570366


**Análise:** Pode-se concluir que o método de gradiente conjugado não linear é mais resistente a valores maiores de n. Todavia, é possível observar que o aumento do tempo necessário de computação aumenta consideravelmente com o aumento do n.

## Conclusão

Pelos experimentos conduzidos, podemos observar que há uma diferença considerável no desempenho dessas técnicas. Em particular, o algoritmo de gradiente conjugados não-linear com fletcher-reeves obteve melhores resultados, principalmente quando consideram-se funções com autovalores mais discrepantes, assim como com funções com $n$ grandes.